<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SPSS Collaboration and Deployment Services

### Compatible with Cloud Pak for Data Only

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
 - Setup
 - Binding machine learning engine
 - Subscriptions
 - Performance monitor, scoring and payload logging
 - Quality monitor and feedback logging
 - Fairness,Drift monitoring and explanations

## Setup

### Sample model creation using SPSS Modeler

- Download training data set from [here](https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/credit_risk_training.csv)
- Download SPSS Modeler stream from [here](https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/Cloud%20Pak%20for%20Data/SPSS%20C%26DS/assets/models/german_credit_risk_tutorial.str)
- Deploy the model using SPSS C&DS as web service

### Installation and authentication

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install --upgrade ibm-watson-openscale | tail -n 1

Import and initiate.

In [ ]:
WOS_CREDENTIALS = {
    "url": "***",
    "username": "***",
    "password": "***"
}

In [ ]:
WML_CREDENTIALS = {
                   "url": "***",
                    "username": "***",
                    "password": "***",
                    "instance_id": "wml_local",
                   "version" : "3.5"

                  }

## Cloud object storage details
In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find COS_API_KEY_ID and COS_RESOURCE_CRN variables in Service Credentials in menu of your COS instance. Used COS Service Credentials must be created with Role parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.
COS_ENDPOINT variable can be found in Endpoint field of the menu.

In [ ]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [ ]:
BUCKET_NAME = "***" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training.csv"

In [ ]:
instance_id='***' ## default instance ID: 00000000-0000-0000-0000-0000000000000000

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],service_instance_id=instance_id,authenticator=authenticator)
wos_client.version


#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS (ICP)
- DATABASE_CREDENTIALS (DB2 on ICP)
- SCHEMA_NAME

In [ ]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [ ]:
SCHEMA_NAME = 'SPSSTF01'

### DataMart setup

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were **not** supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there **unless** there is an existing datamart **and** the **KEEP_MY_INTERNAL_POSTGRES** variable is set to **True**. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

<a id="binding"></a>
## Bind machine learning engines

### Bind  `SPSS C&DS` machine learning engine

Provide credentials using following fields:
- `username`
- `password`
- `url`

In [ ]:
SPSS_CDS_ENGINE_CREDENTIALS = {
        "url": "***",
        "username": "admin",
        "password": "spss",
    }

In [ ]:
SERVICE_PROVIDER_NAME = "V2 SPSS test"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.SPSS_COLLABORATION_AND_DEPLOYMENT_SERVICES,
        credentials=SPSSCredentials(
            url=SPSS_CDS_ENGINE_CREDENTIALS['url'],      
            username=SPSS_CDS_ENGINE_CREDENTIALS["username"],
            password=SPSS_CDS_ENGINE_CREDENTIALS['password']
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result['resources']
asset_deployment_details

In [ ]:
MODEL_NAME='german_credit_risk_tutorial_BiasQA' # use the model name here 
model_asset_details_from_deployment = [asset for asset in asset_deployment_details if asset['entity']["name"]==MODEL_NAME]
source_uid = [asset['entity']['asset']['asset_id'] for asset in asset_deployment_details if asset['entity']["name"]==MODEL_NAME]
if len(model_asset_details_from_deployment)>0:
    [model_asset_details_from_deployment] = model_asset_details_from_deployment
    [source_uid] = source_uid
else:
    raise ValueError('Model with name "{}" not found.'.format(MODEL_NAME))

<a id="subsciption"></a>
## Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [ ]:
wos_client.subscriptions.show()

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == source_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [ ]:
training_data_reference= TrainingDataReference(type='cos',
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = training_data_file_name),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL}))

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=model_asset_details_from_deployment['metadata']['guid'],
            name=model_asset_details_from_deployment['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=model_asset_details_from_deployment['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='Risk',
            probability_fields=['$NP-No Risk','$NP-Risk'],
            prediction_field='$N-Risk',
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=training_data_reference,
            input_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["input_data_schema"]),
            output_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["output_data_schema"])

        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

## Performance monitor, scoring and payload logging

### Score the credit risk model and measure response time

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import time
import json

scoring_endpoint = subscription_details.to_dict()['entity']['deployment']['url']
input_table_id = subscription_details.to_dict()['entity']['asset_properties']['input_data_schema']['id']
node_id = subscription_details.to_dict()['entity']['asset']['name']

scoring_payload = {'requestInputTable': [{'id': input_table_id, 'requestInputRow': [{'input': [
            {'name': 'CheckingStatus', 'value': '0_to_200'}, {'name': 'LoanDuration', 'value': 31},
            {'name': 'CreditHistory', 'value': 'credits_paid_to_date'}, {'name': 'LoanPurpose', 'value': 'other'},
            {'name': 'LoanAmount', 'value': 1889}, {'name': 'ExistingSavings', 'value': '100_to_500'},
            {'name': 'EmploymentDuration', 'value': 'less_1'}, {'name': 'InstallmentPercent', 'value': 3},
            {'name': 'Sex', 'value': 'female'}, {'name': 'OthersOnLoan', 'value': 'none'},
            {'name': 'CurrentResidenceDuration', 'value': 3}, {'name': 'OwnsProperty', 'value': 'savings_insurance'},
            {'name': 'Age', 'value': 32}, {'name': 'InstallmentPlans', 'value': 'none'},
            {'name': 'Housing', 'value': 'own'}, {'name': 'ExistingCreditsCount', 'value': 1},
            {'name': 'Job', 'value': 'skilled'}, {'name': 'Dependents', 'value': 1},
            {'name': 'Telephone', 'value': 'none'}, {'name': 'ForeignWorker', 'value': 'yes'}]}]}], 'id': node_id}

start_time = time.time()
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))

response_time = int((time.time() - start_time)*1000)
result = resp_score.json()

print(result)

### Store the request and response in payload logging table

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your application so it is logged automatically each time you score the model.

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

In [ ]:
import pandas as pd
df_data = pd.read_csv("https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/Cloud%20Pak%20for%20Data/SPSS%20C%26DS/assets/data/credit_risk_spss/payload_credit_risk.csv")
df_data=df_data.drop(['Risk'],axis=1)
df_data.head()

In [ ]:
#score using couple sample records
payload=df_data.sample(2)

In [ ]:
values = payload.values.tolist()
columns = payload.columns.tolist()

In [ ]:
records_list=[]
for i in payload.to_dict('records'):
    b=[{"name":x,"value":v} for x,v in i.items()]
    records_list.append({'input': b})

In [ ]:
scoring_payload = {'requestInputTable': [{'id': input_table_id, 'requestInputRow': records_list}], 'id': node_id}

In [ ]:
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))

response_time = int((time.time() - start_time)*1000)
result = resp_score.json()
print(result)

### Format scoring response for payload logging

In [ ]:
res_values=[]
for i in result['rowValues']:
    d= [j for j in i['value']]
    res_values.append([k['value'] for k in d])

In [ ]:
dtype_idx=[1,4,7,10,12,15,17] # change numeric features values in scoring response from String to Integer
dtype_predictions_idx=[21,22,23] # change prediction, probability column values in scoring response from String to Float

for val in res_values:
    for idx in dtype_idx:
        val[idx]=int(val[idx])
    for idx in dtype_predictions_idx:
        val[idx]=float(val[idx])     

In [ ]:
request = {
            "fields": payload.columns.tolist(),
            "values": payload.values.tolist()
        }
response = {
            "fields": result['columnNames']['name'],
            "values": res_values
        }

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
               scoring_id=str(uuid.uuid4()),
               request=request,
               response=response,
               response_time=460
           )])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

In [ ]:
wos_client.data_sets.show_records(payload_data_set_id)

## Quality monitor and feedback logging

### Enable quality monitoring

In [ ]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

#### Store feedback using CSV format from file

In [ ]:
#!wget https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spss/credit-risk/data/credit_risk_feedback.csv
feed_data_load = pd.read_csv('https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/Cloud%20Pak%20for%20Data/SPSS%20C%26DS/assets/data/credit_risk_spss/feedback_credit_risk.csv')
feedback_data = json.loads(feed_data_load.to_json(orient='records'))
feedback_data

In [ ]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)

In [ ]:
wos_client.data_sets.show_records(data_set_id=feedback_dataset_id)

### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

## Fairness monitoring and explanations

### Enable and run fairness monitoring

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 40
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id

### Score, format and store payload records

In [ ]:
payload=df_data.sample(50)

In [ ]:
records_list=[]
for i in payload.to_dict('records'):
    b=[{"name":x,"value":v} for x,v in i.items()]
    records_list.append({'input': b})

In [ ]:
scoring_payload = {'requestInputTable': [{'id': input_table_id, 'requestInputRow': records_list}], 'id': node_id}
resp_score = requests.post(url=scoring_endpoint, json=scoring_payload, auth=HTTPBasicAuth(username=SPSS_CDS_ENGINE_CREDENTIALS['username'], password=SPSS_CDS_ENGINE_CREDENTIALS['password']))
result = resp_score.json()

In [ ]:
res_values=[]
for i in result['rowValues']:
    d= [j for j in i['value']]
    res_values.append([k['value'] for k in d])

In [ ]:
dtype_idx=[1,4,7,10,12,15,17] # change numeric features values in scoring response from String to Integer
dtype_predictions_idx=[21,22,23] # change prediction, probability column values in scoring response from String to Float

for val in res_values:
    for idx in dtype_idx:
        val[idx]=int(val[idx])
    for idx in dtype_predictions_idx:
        val[idx]=float(val[idx])
len(res_values)       

In [ ]:
request = {
            "fields": payload.columns.tolist(),
            "values": payload.values.tolist()
        }
response = {
            "fields": result['columnNames']['name'],
            "values": res_values
        }

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
               scoring_id=str(uuid.uuid4()),
               request=request,
               response=response,
               response_time=460
           )])
time.sleep(10)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(10)
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

### Explainability configuration and run

#### Enable explainability

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

#### Get sample transaction_id from payload logging table (`scoring_id`)

In [ ]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

In [ ]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

### Enable and run drift monitoring

In [ ]:
!rm -rf creditrisk_spss_drift_detection_model.tar.gz
!wget -O creditrisk_spss_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/draft_v1/Cloud%20Pak%20for%20Data/SPSS%20C%26DS/assets/models/spss_creditrisk_drift_detection_model.tar.gz?raw=true 

In [ ]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_spss_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 40,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit SPSS model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

---